In [24]:
import pandas as pd
import numpy as np
from typing import List
import uvicorn


In [4]:
def calculate_npv(
    cash_flows: List[float],
    discount_rate: float,
    energy_savings: float,
    initial_investment: float,
    lifetime: int
) -> float:
    """
    Calculate the Net Present Value (NPV) for a series of cash flows.
    
    - **cash_flows**: List of floats, e.g. [-1000, 300, 400, 500]
    - **discount_rate**: Decimal, e.g. 0.1 for 10%
    - **energy_savings**: Float representing annual energy savings
    - **initial_investment**: Float representing the initial investment cost
    - **lifetime**: Integer representing the lifetime of the investment in years
    """
    npv = -initial_investment
    for t in range(1, lifetime + 1):
        # guard against index errors if cash_flows list is shorter than lifetime
        cf = cash_flows[t-1] if t-1 < len(cash_flows) else 0.0
        npv += (cf + energy_savings) / ((1 + discount_rate) ** t)
    
    return npv

In [7]:
cash_flows= [1000.0, 1200.5, 1500.25, 1100]
discount_rate= 0.08
energy_savings= 100.0
initial_investment= -3000.0
lifetime= 4

calculate_npv(
    cash_flows=cash_flows,  
    discount_rate=discount_rate,
    energy_savings=energy_savings,
    initial_investment=initial_investment,
    lifetime=lifetime
)

7285.853521369258

In [82]:
def calculate_ii(
    capex: float,
    interest_rate: float = 0.0,
    loan_term: float = 0.0,
    loan_amount: float = 0.0,
    subsidy: float = 0.0,
) -> float:
    """
    Calculate the Initial Investment (II) based on provided inputs.

    - Case 1: Only CAPEX
    - Case 2: CAPEX with Subsidy
    - Case 3: CAPEX with Loan
    - Case 4: CAPEX with Subsidy and Loan

    Missing values (like loan or subsidy) are treated as 0.
    """

    if subsidy == 0 and loan_amount == 0:
        # Case 1: Only CAPEX
        ii = capex

    elif subsidy > 0 and loan_amount == 0:
        # Case 2: CAPEX with Subsidy
        ii = capex - subsidy

    elif loan_amount > 0 and subsidy == 0:
        # Case 3: CAPEX with Loan
        ii = capex - loan_amount

    elif loan_amount > 0 and subsidy > 0:
        # Case 4: CAPEX with Subsidy and Loan
        ii = capex - subsidy - loan_amount

    else:
        # Fallback 
        ii = capex

    return ii

In [84]:
capex = 1000
interest_rate = 0.1
loan_term = 10
subsidy = 2000


calculate_ii(
    capex=capex,
    interest_rate=interest_rate,
    loan_term=loan_term,
    subsidy=subsidy
)

-1000

In [41]:
def calculate_opex(
    energy_mix: List[float],
    energy_prices: List[float],
    maintenance_cost: float,
) -> float:


    opex = []
    for t in range(len(energy_mix)):
        if t < len(energy_prices):
            opex.append(energy_mix[t] * energy_prices[t]) 
        else:
            opex.append = 0  # Default to maintenance cost if no price data available

    opex = np.sum(opex) + maintenance_cost
    

    return float(opex)

In [42]:
energy_mix = [0.3, 0.4, 0.2, 0.1]
energy_prices = [1000, 1200, 1500, 1100]
maintenance_cost = 100

calculate_opex(
    energy_mix=energy_mix,
    energy_prices=energy_prices,
    maintenance_cost=maintenance_cost
)



1290.0

In [99]:
def calculate_roi(
        capex: float= 0.0,
        interest_rate: float= 0.0,
        loan_term: float= 0.0,
        loan_amount: float= 0.0,
        subsidy: float= 0.0,
        energy_savings: float= 0.0,
        energy_mix: List[float]=None,
        energy_prices: List[float]=None,
        maintenance_cost: float= 0.0,
        other_outflows: float= 0.0,
)-> float: 
    


    if energy_mix is None:
        energy_mix=[]

    if energy_prices is None:
        energy_prices=[]
    #Calculate roi
    #first calucalte opex and initial investment

    #OPEX

    opex = []

    for t in range(len(energy_mix)):
        if t < len(energy_prices):
            opex.append(energy_mix[t] * energy_prices[t]) 
        else:
            opex.append(0)  # Default to maintenance cost if no price data available

    opex = np.sum(opex) + maintenance_cost
    opex=float(opex)  # Ensure opex is a float

    #Initial Investment (II)

    if subsidy == 0 and loan_amount == 0:
        # Case 1: Only CAPEX
        ii = capex

    elif subsidy > 0 and loan_amount == 0:
        # Case 2: CAPEX with Subsidy
        ii = capex - subsidy

    elif loan_amount > 0 and subsidy == 0:
        # Case 3: CAPEX with Loan
        ii = capex - loan_amount

    elif loan_amount > 0 and subsidy > 0:
        # Case 4: CAPEX with Subsidy and Loan
        ii = capex - subsidy - loan_amount

    else:
        # Fallback 
        ii = capex


#ROI
    
    net_profit= energy_savings-opex-other_outflows
    if ii == 0:
        roi = 0.0
    else:
     roi = (net_profit-ii)/ii *100

    roi=float(roi)

    return roi

In [100]:
capex = 1000
loan_amount = 200
subsidy = 100
energy_savings = 1500
energy_mix = [100, 100, 100]
energy_prices = [2, 2, 2]
maintenance_cost = 50
other_outflows = 100

calculate_roi(
    capex=capex,
    loan_amount=loan_amount,
    subsidy=subsidy,
    energy_savings=energy_savings,
    energy_mix=energy_mix,
    energy_prices=energy_prices,
    maintenance_cost=maintenance_cost,
    other_outflows=other_outflows
)

7.142857142857142

In [109]:
def calculate_irr(
        
        capex: float = 0.0,
        interest_rate: float = 0.0,
        loan_term: float = 0.0,
        loan_amount: float = 0.0,
        subsidy: float = 0.0,
        energy_savings: float = 0.0,
        energy_mix: List[float] = None,
        energy_prices: List[float] = None,
        maintenance_cost: float = 0.0,
        other_outflows: float = 0.0,
        project_lifetime: float = 20.0,  # Default project lifetime
) -> float:
    
    if energy_mix is None:
        energy_mix = []

    if energy_prices is None:
        energy_prices = []

    # Calculate OPEX
    opex = []
    for t in range(len(energy_mix)):
        if t < len(energy_prices):
            opex.append(energy_mix[t] * energy_prices[t])
        else:
            opex.append(maintenance_cost)  # Default to maintenance cost if no price data available

    opex_total = np.sum(opex) + maintenance_cost

    # Calculate Initial Investment (II)
    ii = capex - subsidy - loan_amount if (subsidy > 0 or loan_amount > 0) else capex

    if ii==0:
        irr =0
    else:
    # Calculate IRR using a simple formula (this is a placeholder, actual IRR calculation may require more complex financial modeling)
     irr = (energy_savings - opex_total - other_outflows) / ii

    irr=float(irr)

    return irr

In [110]:
capex = 0
loan_amount = 0
subsidy = 0
energy_savings = 0
energy_mix = [100, 100, 100]
energy_prices = [2, 2, 2]
maintenance_cost = 0
other_outflows = 0
project_lifetime = 0

calculate_irr(
    capex=capex,
    loan_amount=loan_amount,    
    subsidy=subsidy,
     energy_savings = energy_savings,
    energy_mix = energy_mix,
energy_prices = energy_prices,
maintenance_cost = maintenance_cost ,
other_outflows = other_outflows,
project_lifetime = project_lifetime,
)

0.0